In [1]:

from splinter import Browser
from bs4 import BeautifulSoup
import time
import pymongo
import requests
from splinter.exceptions import ElementDoesNotExist
import pandas as pd


In [2]:


def init_browser():

        executable_path = {"executable_path": "chromedriver.exe"}
        return Browser("chrome", **executable_path, headless=False)




In [ ]:
address = 'Princess Pl'
browser = init_browser()
url = 'https://property.spatialest.com/nc/durham/#/'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

results = soup.find_all('input', class_='search')
print(results)

In [ ]:
browser.find_by_id('primary_search').first.fill(address)

In [ ]:
browser.click_link_by_partial_href('#') 

In [ ]:
# browser.click_link_by_partial_href("#/property")

In [ ]:
# property_link = browser.html
# prop_link = BeautifulSoup(property_link, 'html.parser')


In [ ]:
# prop_mailing = prop_link.find('div', class_='mailing')
# prop_mailing.text

In [ ]:
# sales = []
# prop_table = prop_link.findAll('table')[3]
# for tr in prop_table.findAll('tr'):
#     for td in tr.findAll('td'):
#         sales.append(td.text)
# sales[0]

In [ ]:
# browser.click_link_by_partial_text('sults') 

In [ ]:
data = browser.url
browser.visit(data)

property_link1 = browser.html
prop_link1 = BeautifulSoup(property_link1, 'html.parser')
prop_link1
prop_list = []
addresses = prop_link1.findAll('div', class_="item-1")
for address in addresses:
    prop_list.append(address.text)
prop_list

In [ ]:
browser.click_link_by_partial_text('Next') 

In [ ]:
data = browser.url
browser.visit(data)
property_link2 = browser.html
prop_link2 = BeautifulSoup(property_link2, 'html.parser')
prop_link2

addresses1 = prop_link2.findAll('div', class_="item-1")
for address1 in addresses1:
    prop_list.append(address1.text)
prop_list

In [ ]:
browser.click_link_by_partial_href('spatialest')
time.sleep(3)
prop_mailing1 = []    
sales = []
for prop in prop_list:
    browser.find_by_id('primary_search').first.fill(prop)
    time.sleep(3)
    browser.click_link_by_partial_href('#') 
    time.sleep(5)
    data = browser.url
    browser.visit(data)
    time.sleep(3)
    property_link3 = browser.html
    prop_link = BeautifulSoup(property_link3, 'html.parser')
    prop_mailing = prop_link.find('div', class_='mailing')
    prop_mailing1.append(prop)
    sales.append(prop)
    prop_table = prop_link.findAll('table')[3]
    for tr in prop_table.findAll('tr'):
        for td in tr.findAll('td'):
            sales.append(td)
    browser.click_link_by_partial_href('spatialest')
    time.sleep(3)

In [ ]:
sales1 = []
for x in sales:
    sales1.append(x.text)
sales1


In [ ]:
len(sales1)

In [ ]:
len(prop_list)

In [ ]:
print(sale_price)
print(sale_date)
sale_price = sales1[::2]
sale_date =  sales1[1::2]


In [ ]:
df = pd.DataFrame(columns=['Address', 'Price', 'Date'])
# df['Address']=prop_list
df['Price']=sale_price
df['Date']=sale_date
df

In [ ]:
len(prop_list)